In [ ]:

#download the train, test and validate sets
!wget https://www.dropbox.com/scl/fi/em9gg01fixe0evr2gm1yr/train.zip?rlkey=82gu4jxpee8ud3xfhkcwobu9l&dl=0 # train
!wget https://www.dropbox.com/scl/fi/toycvi2k3h0dz42gl3vk0/test.zip?rlkey=6wjn1yvhcfiuymbhkc2ubcro7&dl=0 # test
!wget https://www.dropbox.com/scl/fi/khtr33xeykwpjjezlbkdd/validation.zip?rlkey=zbpzzxv3c368df70nrhssi3sz&dl=0 # validation

In [ ]:
# unzip the downloaded zip files
!unzip /content/train.zip?rlkey=82gu4jxpee8ud3xfhkcwobu9l # train
!unzip /content/test.zip?rlkey=6wjn1yvhcfiuymbhkc2ubcro7 # test
!unzip /content/validation.zip?rlkey=zbpzzxv3c368df70nrhssi3sz # validation

In [ ]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
train_dir = '/content/train'
test_dir = '/content/test'
valid_dir = '/content/validation'

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.3)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

valid_generator = train_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_height,img_width),
    batch_size = batch_size,
    class_mode='categorical',
    subset='validation')

Found 840 images belonging to 3 classes.
Found 90 images belonging to 3 classes.


In [ ]:
test_generator = train_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode='categorical',
    subset='validation')

Found 90 images belonging to 3 classes.


In [ ]:
x,y = test_generator.next()
x.shape

(1, 224, 224, 3)

In [ ]:
base_model =  ResNet50(include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])
model.fit(train_generator,
epochs = 20)

94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/20
23/23 [==============================] - 31s 935ms/step - loss: 0.7727 - accuracy: 0.8417
Epoch 2/20
23/23 [==============================] - 22s 950ms/step - loss: 0.1167 - accuracy: 0.9679
Epoch 3/20
23/23 [==============================] - 22s 939ms/step - loss: 0.0433 - accuracy: 0.9786
Epoch 4/20
23/23 [==============================] - 22s 965ms/step - loss: 0.0254 - accuracy: 0.9905
Epoch 5/20
23/23 [==============================] - 22s 952ms/step - loss: 0.0386 - accuracy: 0.9869
Epoch 6/20
23/23 [==============================] - 22s 940ms/step - loss: 0.0165 - accuracy: 0.9952
Epoch 7/20
23/23 [==============================] - 22s 950ms/step - loss: 0.0143 - accuracy: 0.9929
Epoch 8/20
23/23 [==============================] - 21s 900ms/step - loss: 0.0122 - accuracy: 0.9964
Epoch 9/20
23/23 [==============================] - 22s 985ms/step - loss: 0.0049 - accuracy: 1.0000
Epoch 10/20
23/23 [=======

In [ ]:
model.save('disaster_resnet50.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
predictions = model.predict(test_generator, steps=len(test_generator), verbose=1)
y_pred = np.argmax(predictions, axis=1)
y_true = np.argmax(test_labels, axis=1)

90/90 [==============================] - 3s 27ms/step


NameError: name 'test_labels' is not defined